# Synthetic Recurrent Neural Networks (RNN) using Basic Python Packages with comparison of the Actual API RNN

## Importing the required Libraries

In [1]:
from tensorflow.keras.layers import SimpleRNN, Dense, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Number_Of_Samples = 100
Dimensions = 3
Hidden_Units = 6
Sequqnce_Length = 15
Outputs = 5

data = np.random.randn(Number_Of_Samples, Sequqnce_Length, Dimensions)

In [3]:
data.shape

(100, 15, 3)

## Constructing a RNN from API

In [4]:
inputShape = Input(shape = (Sequqnce_Length, Dimensions))
RNNModel_From_API = SimpleRNN(Hidden_Units)(inputShape)
RNNModel_From_API = Dense(Outputs)(RNNModel_From_API)

RNNModel_From_API = Model(inputShape, RNNModel_From_API)

In [5]:
RNNModel_From_API.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15, 3)]           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 6)                 60        
_________________________________________________________________
dense (Dense)                (None, 5)                 35        
Total params: 95
Trainable params: 95
Non-trainable params: 0
_________________________________________________________________


In [6]:
data[0].shape

(15, 3)

In [7]:
RNNModel_From_API.predict(data)[0]

array([-0.29195532, -0.14294672,  0.10680032, -0.43692708,  0.58487356],
      dtype=float32)

### Checking SimpleRNN Layer aprameters & their Shapes

In [8]:
## Layers in the Model
RNNModel_From_API.layers

In [9]:
p, q, r = RNNModel_From_API.layers[1].get_weights()
print("Shape of Weights of Input for Hidden Layer {} \nShape of Weights of Hidden Parameters for Hidden Layers {}\nShape of the Biases for the Hidden Layer {}".format(
    p.shape, q.shape, r.shape))

Shape of Weights of Input for Hidden Layer (3, 6) 
Shape of Weights of Hidden Parameters for Hidden Layers (6, 6)
Shape of the Biases for the Hidden Layer (6,)


In [10]:
RNNModel_From_API.layers[2].get_weights()

[array([[ 0.24443406,  0.57757634,  0.66978115, -0.09970021,  0.6720875 ],
        [ 0.4150825 , -0.5342384 ,  0.41841716,  0.37949103,  0.05946559],
        [-0.4465566 , -0.0190466 ,  0.2963224 , -0.53669405,  0.3853758 ],
        [-0.39801222, -0.42716607, -0.3585383 ,  0.30239218,  0.0600062 ],
        [-0.25734153, -0.42462218, -0.4251675 , -0.12579215,  0.7083252 ],
        [ 0.09524584,  0.20559543,  0.5010094 ,  0.25290245, -0.35680544]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [11]:
a, b = RNNModel_From_API.layers[2].get_weights()
print("Shape of Weights for the output Layer {} \nShape of Biases for the Output Layer {}".format(a.shape, b.shape))

Shape of Weights for the output Layer (6, 5) 
Shape of Biases for the Output Layer (5,)


In [12]:
data[0].shape

(15, 3)

## Creating Synthetic RNN

***Wx = Wieghts of the Input in Hidden Layer***

***Wh = Weights of the Hidden Parameters in the Hidden Layer***

***Bh = Biases of the Hidden Parameters in the Hidden Layer***

***Wo = Weights of the Output Parameters in the Output Layer***

***Bo = Biases of the Output Parameters in the Output Layer***

### Retrieving the required Wieghts

In [13]:
Wx, Wh, Bh = RNNModel_From_API.layers[1].get_weights()
Wo, Bo = RNNModel_From_API.layers[2].get_weights()


In [14]:
sampleData = data[0]
hiddenState_Previous = np.zeros(Hidden_Units)
output = []

for sequence in range(Sequqnce_Length):
    hiddenParameter = np.tanh( sampleData[sequence].dot(Wx) + hiddenState_Previous.dot(Wh) + Bh )
    y = hiddenParameter.dot(Wo) + Bo
    
    output.append(y)
    hiddenState_Previous = hiddenParameter

print(output[-1])

[-0.2919553  -0.1429467   0.10680034 -0.43692709  0.58487372]


In [15]:
print(RNNModel_From_API.predict(data)[0])

[-0.29195532 -0.14294672  0.10680032 -0.43692708  0.58487356]


### The above shown parameters are exactly same, therefore this proves that the code written above is the exact code for the functioning of the RNN

## Another way of writing the exact code

In [16]:
sampleData = data[0]
hiddenState_Previous = np.zeros(Hidden_Units)
output = []

for sequence in range(Sequqnce_Length):
    hiddenParameter = np.tanh( np.transpose(Wx).dot(sampleData[sequence])
                              + np.transpose(Wh).dot(hiddenState_Previous) + Bh )
    y = hiddenParameter.dot(Wo) + Bo
    
    output.append(y)
    hiddenState_Previous = hiddenParameter

print(output[-1])

[-0.2919553  -0.1429467   0.10680034 -0.43692709  0.58487372]


In [17]:
Wx.shape, sampleData.shape, Wo.shape, Wh.shape, hiddenState_Previous.shape

((3, 6), (15, 3), (6, 5), (6, 6), (6,))

### The above calculated results also are exactly same as the aforementioned results, this is because the way of writing code is different, but internally it means exactly the same.

***From here, it can be concluded that the equation for RNN is `hiddenParameter(i) = Tanh(Wx(Transpose) * x(i)   + Wh(Transpose) * hiddenParameter(i - 1)  + Bh) `  & `y(i) = Wo(Transpose) * hiddenParameter(i) + Bo` !!!***

## Explanation of the Trainable Parameters in SimpleRNN

In [18]:
RNNModel_From_API.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15, 3)]           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 6)                 60        
_________________________________________________________________
dense (Dense)                (None, 5)                 35        
Total params: 95
Trainable params: 95
Non-trainable params: 0
_________________________________________________________________


***SimpleRNN layer consists only the parameters of the HiddenLayers of the RNN, where as the Dense layer consists the parameters of the output Layer!***
  * Explanation for SimpleRNN Layer:
  
    * Here, the toatal parameters are obtained by the combination of the parameters trained in the equation of HiddenLayer as mentioned above.
    <br>
    * For the first part, i.e. Input Part, total number of parameters trained are equal to Dimension x HiddenUnits, in present case: 3 x 5 = 15 Parameters.
    <br>
    
    * Then there is turn of Hidden Parameters, it directly depends only on the HiddenUnits, and total parameters are HiddenUnits * HiddenUnits, in present case: 6 x 6 = 36 Parameters. It is because, the hidden layer do not calculares only a single value, every time a matrix of values/weights are calcualted, & based on the HiddenUnits, sequence division is done, & each iteration, the shape of the Weights are  Dimension of **HiddenUnits * HiddenUnits**, whereas the shape of the inputData in each iteration of Loop is **HiddenUnits * 1**, which when multiplied, provides a matrix of shape **HiddenUnits * 1**. Maximum number of parameters which are to be trained in this part are HiddenUnits * HiddenUnits, therefore, in present case: 6 * 6 = 36.
      * The reason why, shape of Weights generated i.e. shape of Wh is HiddenUnits * HiddenUnits, is that the previous weights i.e. Wx(shape: Dimensions * HiddenUnits) when Transposed & multiplied with Input(shape: Dimension * 1), output matrix is of shape HiddenUnits * 1. Now, to add another matrix uniformly to a matrix of the shape HiddenUnits * 1, the shape of that matrix should also be HiddenUnits * 1, otherwise addition will not be possible.